# Первое задание.
## Работу выполнил: Шурыгин Антон, Б01 - 909.

In [1]:
! pip install kaggle
! pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 4.5 MB/s 


### Подтягиваем датасет с kaggle

In [2]:
!mkdir -p ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"uslsteen", "key":"56dbf05759db832787aa95054b6b5a45"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle competitions download -c fall-ml2-mipt-2022 --force

 76% 30.0M/39.3M [00:00<00:00, 160MB/s] 
100% 39.3M/39.3M [00:00<00:00, 138MB/s]


In [4]:
! unzip /content/fall-ml2-mipt-2022.zip

Archive:  /content/fall-ml2-mipt-2022.zip
  inflating: sample.csv              
  inflating: test.csv                
  inflating: train.csv               


### Подключаем небоходимые модули

In [5]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torchvision
from torchvision import transforms, models
import torchmetrics
import torch.nn.functional as F
import torchmetrics.functional as FM
from torch.utils.data import DataLoader
import copy

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

### Определим класс **DataWrapper** - класс владеющий и управляющий загруженным датасетом.

In [6]:
class DataWrapper():
  def __init__(self, train_path : str = str(), test_path : str = str(),  feature_vars : list = list(), target_vars : list = list()):
  #
    self.feature_vars = feature_vars
    self.target_vars = target_vars
    self.all_vars = feature_vars + target_vars
    #
    train_ds = pd.read_csv(train_path)
    test_ds = pd.read_csv(test_path)
    #
    (self.feature_train, self.target_train) = slice_data(train_ds, feature_vars, target_vars)
    self.feature_test = slice_data(test_ds, feature_vars) 
    # 
    self.dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.scaler = StandardScaler()
    #
  #
  def cast(self, data, is_target : bool = True):
  #  
    casted_data = None
    #
    if is_target == True:
      data = data.reshape(-1, 1, 10, 10) * 10
      data = np.repeat(data, 3, axis=1)
      casted_data = torch.tensor(data).float().to(self.dev)
    else:
      casted_data = torch.tensor(data).type(torch.LongTensor).to(self.dev)
    #
    return casted_data
    #
  #
  def split_data(self):
    X_train, X_test, Y_train, Y_test = split(self.feature_train, self.target_train, 0.1, 3)
    return X_train, X_test, Y_train, Y_test
    #
  #
  def make_preproc(self, is_train : bool = True):
  #
    if is_train == True:
      X_train, X_test, Y_train, Y_test = self.split_data()
      #
      self.scaler.fit(X_train)
      #
      X_train = self.scaler.transform(X_train)
      X_train = self.cast(X_train)
      Y_train = self.cast(Y_train, is_target = False)
      #
      X_test = self.scaler.transform(X_test)
      X_test = self.cast(X_test)
      Y_test = self.cast(Y_test, is_target = False)
      return X_train, Y_train, X_test, Y_test
    # NOTE: if not train preprocessing
    else: 
      X_test = self.scaler.transform(self.feature_test)
      X_test = self.cast(X_test)
      return X_test 
    #
  # NOTE: custom feature data scaller
  def scale_features(self):
    sc_feature = StandardScaler()
    #
    self.features_test = sc_feature.fit_transform(self.features_test)
    self.features_train = sc_feature.transform(self.features_train)
    #
  #
  def scale_target(self):
    sc_target = StandardScaler()
    #
    self.target_test = sc_target.fit_transform(self.target_test)
    self.target_train = sc_target.transform(self.target_train)
    #
  #
def slice_data(data, features_slice : list = list(), target_slice : list = list()):
  if target_slice != list():
    return data.loc[:, features_slice[0]:features_slice[1]].to_numpy(), data.loc[:, target_slice[0]].to_numpy()
  else:
    return data.loc[:, features_slice[0]:features_slice[1]].to_numpy()
  #
#
# NOTE: custom dataset splitter
def split(X, Y, testsize : float, random_st : int):
#
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = testsize, random_state=random_st)
  return X_train, X_test, Y_train, Y_test
  #
#

- Определим СustomDataset -- прокси - класс для хранения для датасета.
- Наследуемся от ```torch.utils.data.Dataset``` для последующей передачи в конструктор ```DataLoader```

In [7]:
class СustomDataset(torch.utils.data.Dataset):
  def __init__(self, feature, target):
    self.feature = feature
    self.target = target

  def __len__(self):
    return len(self.target)
  
  def __getitem__(self, ind):
    return self.feature[ind], self.target[ind]

- Cоздаем wrapper, загружаем нужные .csv файлы и достаем признаки ```[x_0, x_99]```, a так же таргет - переменную - ```Category```
- Количество объектов признаков в размере 100 - результат перебора разных размеров датасета. 

In [8]:
path = '/content/'
wrapper = DataWrapper(path + 'train.csv', path + 'test.csv', ['x_0', 'x_99'], ['Category'])

### Далее - препроцессинг данных.
- Дефолтный булевый флаг высталвен в True => работаем с train.csv датасаетом.
- Сплитом разбиваем данные и кастуем все нужно к ```torch.tensor``` (а именно feature_train, target_train и feature_test, target_test - numpy матрицы, взятые непосредственно из train.csv датасета).
- Дамп размеров полученных тензоров для самопроверки.

In [9]:
target_train, feature_train, target_test, feature_test = wrapper.make_preproc()
#
print(f"size = {target_train.size()}")
print(f'size = {feature_train.size()}')
print(f"size = {target_test.size()}")
print(f'size = {feature_test.size()}')

size = torch.Size([9000, 3, 10, 10])
size = torch.Size([9000])
size = torch.Size([1000, 3, 10, 10])
size = torch.Size([1000])


- Разбитые на train и test данные отправляем в конструктор кастомного датасета
- А затем отправляем в конструктор DataLoader, чтобы получить нативный доступ к преобразованным семплам.

In [10]:
train_ds = СustomDataset(target_train, feature_train)
test_ds = СustomDataset(target_test, feature_test)

train_dataloader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=1000, shuffle=False)

### Определяем кастомный классификатора.
- Изначально выбор resnet34 в качестве модели объясняется огромным количеством признаков. 
- После шаманства с количеством признаков объектов и их уменьшением в конечном итоге было принято решение оставить как есть. Пусть, это, вероятно и оверхед - использование нейронки для изображений в данном случае.

In [11]:
class Classifier(torch.nn.Module):
#
  def __init__(self):
    super().__init__()
    self.model = models.resnet34(num_classes=32)
    self.accuracy = torchmetrics.Accuracy('multiclass', num_classes=32)
    self.learning_rate = 1e-3
    #
  #
  def forward(self, X):
    return self.model(X)
    #
  #
  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), self.learning_rate)

In [12]:
def fit(classifier, train_dataloader, test_dataloader, train_ds : СustomDataset, test_ds : СustomDataset,
                    optimizer, max_epochs=10, device = None):
  classifier.to(device)

  opt_model = classifier
  opt_acc = 0

  for epoch in range(0, max_epochs):
    classifier.train()
    for i, (features, label) in enumerate(train_dataloader):
      features, label = features.to(device), label.to(device)
      optimizer.zero_grad()
      pred = classifier(features)
      loss = F.cross_entropy(pred, label)
      loss.backward()
      optimizer.step()

    train_accur = classifier.accuracy(torch.argmax(classifier(train_ds.feature), 1), 
                                         torch.tensor(train_ds.target).to(device))
    #
    test_accur = classifier.accuracy(torch.argmax(classifier(test_ds.feature), 1), 
                                         torch.tensor(test_ds.target).to(device))
    
    if (test_accur > opt_acc):
      opt_model = classifier
      opt_acc = test_accur

    print(f"[{epoch + 1}/{max_epochs}]: train accur. = {train_accur}, test accur. = {test_accur}")
  
  return opt_model

In [13]:
classifier = Classifier()
optimizer = classifier.configure_optimizers()

model = fit(classifier, train_dataloader, test_dataloader, train_ds, test_ds, optimizer, max_epochs = 50, device = wrapper.dev)

<ipython-input-12-156a41c870cc>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(train_ds.target).to(device))
<ipython-input-12-156a41c870cc>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(test_ds.target).to(device))


[1/50]: train accur. = 0.18244443833827972, test accur. = 0.16300000250339508
[2/50]: train accur. = 0.2508888840675354, test accur. = 0.23899999260902405
[3/50]: train accur. = 0.32422223687171936, test accur. = 0.2720000147819519
[4/50]: train accur. = 0.363444447517395, test accur. = 0.3140000104904175
[5/50]: train accur. = 0.36133334040641785, test accur. = 0.3059999942779541
[6/50]: train accur. = 0.4020000100135803, test accur. = 0.33399999141693115
[7/50]: train accur. = 0.42311111092567444, test accur. = 0.3659999966621399
[8/50]: train accur. = 0.4234444499015808, test accur. = 0.35899999737739563
[9/50]: train accur. = 0.4372222125530243, test accur. = 0.36399999260902405
[10/50]: train accur. = 0.4487777650356293, test accur. = 0.367000013589859
[11/50]: train accur. = 0.4663333296775818, test accur. = 0.38499999046325684
[12/50]: train accur. = 0.460999995470047, test accur. = 0.3880000114440918
[13/50]: train accur. = 0.4714444577693939, test accur. = 0.4020000100135803
[

In [14]:
_, _, _, Y_test = wrapper.split_data()
#
model.accuracy(torch.argmax(model(target_test), 1), torch.tensor(Y_test).to(wrapper.dev))

tensor(0.4080, device='cuda:0')

### Дампим результаты

In [15]:
def dump(model):
  feature_test = wrapper.make_preproc(is_train = False)
  preds = torch.argmax(model(feature_test), 1).cpu().numpy()

  with open('answer.csv', 'w') as output:
    output.write('Id,Category')
    for id, prediction in enumerate(preds):
      output.write(f'\n{id},{int(prediction)}')

In [16]:
dump(model)